In [1]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models.TorchAvitm import TorchAvitm
from gensim.models import Word2Vec

In [2]:
# Load dataset
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_10")

True

In [3]:
# Load model
model = TorchAvitm()

In [4]:

# Set model hyperparameters
model.hyperparameters['prior_variance'] = 0.2
model.hyperparameters['n_components'] = 5
model.hyperparameters['num_epochs'] = 1

In [5]:
model.hyperparameters

{'prior_variance': 0.2, 'n_components': 5, 'num_epochs': 1}

In [6]:
model.partitioning(True)

In [7]:
model.train_model(dataset, model.hyperparameters)

Settings: 
               N Components: 5
               Topic Prior Mean: 0.0
               Topic Prior Variance: 0.2
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.002
               Momentum: 0.99
               Reduce On Plateau: False
               Save Dir: None
Epoch: [1/1]	Samples: [10758/10758]	Train Loss: 572.320676237306	Time: 0:00:03.378516


{'topics': [['dangerous',
   'alternative',
   'describe',
   'economic',
   'recommend',
   'burn',
   'son',
   'soldier',
   'course',
   'jump'],
  ['anywhere',
   'fault',
   'secret',
   'originally',
   'tie',
   'topic',
   'shift',
   'element',
   'completely',
   'operate'],
  ['try',
   'way',
   'know',
   'think',
   'thing',
   'still',
   'be',
   'get',
   'also',
   'well'],
  ['thing',
   'mean',
   'day',
   'would',
   'way',
   'right',
   'much',
   'issue',
   'time',
   'even'],
  ['originator',
   'unable',
   'setup',
   'direction',
   'solid',
   'component',
   'mountain',
   'huge',
   'defend',
   'bottom']],
 'topic-word-matrix': array([[-0.14488226, -0.14759812,  0.02190099, ...,  0.01369622,
          0.02865248,  0.00538214],
        [ 0.04971226, -0.01680202, -0.060491  , ..., -0.06207202,
         -0.1380107 ,  0.04873045],
        [-0.04562013, -0.01280217, -0.02776971, ..., -0.05532337,
         -0.01347484, -0.00765304],
        [-0.0362308 , -0

In [6]:
# Topic diversity
topic_diversity = Topic_diversity()

# KL_Uniform
#kl_uniform = KL_uniform()

In [7]:
# Define optimization parameters
opt_params = {}
opt_params["n_calls"] = 30
opt_params["minimizer"] = forest_minimize
opt_params["different_iteration"] = 3
opt_params["n_random_starts"] = 5
#opt_params["extra_metrics"] = [kl_uniform] # List of extra metrics
opt_params["n_jobs"] = mp.cpu_count() -1 # Enable multiprocessing
opt_params["verbose"] = True
opt_params["save_path"] = "results" #create folder if it doesn't exist 

In [8]:
# Create search space for optimization
search_space = {
    "num_epochs": Integer(low=1, high=50),
    #"eta": Real(low=0.01, high=5.0)
}

In [9]:
# Initialize optimizer
optimizer = Optimizer(
    model,
    dataset,
    topic_diversity,
    search_space,
    opt_params)

In [ ]:
# Disable computing of topic document matrix to optimize performance
optimizer.topic_document_matrix = False
optimizer.topic_word_matrix = False

In [ ]:
# Optimize
res = optimizer.optimize()

print(res.hyperparameters) # Best values for the hyperparameters
print(res.function_values) # Score of the optimized metric
print("Optimized metric: "+res.optimized_metric)